In [ ]:
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv1D

**Downloading and Extracting Data**

In [ ]:
# Set the Kaggle API credentials path
os.environ['KAGGLE_CONFIG_DIR'] = '/content/kaggle'

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

 99% 2.28G/2.29G [00:21<00:00, 83.2MB/s]
100% 2.29G/2.29G [00:21<00:00, 113MB/s] 


In [ ]:
import zipfile

zip_file_path = '/content/chest-xray-pneumonia.zip'

extracted_folder_path = '/content/dataset'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

**Preprocessing Image Files**

In [ ]:
train_dir = '/content/dataset/chest_xray/train'
train_pneumonia_dir = os.path.join(train_dir, 'PNEUMONIA')
train_normal_dir = os.path.join(train_dir, 'NORMAL')

In [ ]:
img_size = (224, 224)

**feature Extraction using vgg16**

In [ ]:
# Define the arrays to store the features and labels
features = []
labels = []

In [ ]:
model = VGG16(weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Loop through the images and predict features
for img_path in os.listdir(train_pneumonia_dir):
    img = image.load_img(os.path.join(train_pneumonia_dir, img_path), target_size=img_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    features.append(feature)
    labels.append(1)  # Pneumonia class label

for img_path in os.listdir(train_normal_dir):
    img = image.load_img(os.path.join(train_normal_dir, img_path), target_size=img_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    features.append(feature)
    labels.append(0)  # Normal class label

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 20ms/step


In [ ]:
features = np.array(features)
labels = np.array(labels)

In [ ]:
print(labels)

[1 1 1 ... 0 0 0]


In [ ]:
import shutil

# Path to the file you want to copy
source_file = '/content/vgg16_features.npy'

# Destination folder where you want to copy the file
destination_folder = '/content/drive/MyDrive/Pneumonia'

# Check if the source file exists
if os.path.exists(source_file):
    # Copy the file to the destination folder
    shutil.copy(source_file, destination_folder)
    print(f"File '{os.path.basename(source_file)}' copied to '{destination_folder}'")
else:
    print("Source file does not exist.")

File 'vgg16_features.npy' copied to '/content/drive/MyDrive/Pneumonia'


**Training**

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=features.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 1)                 0         
                                                                 
 dense (Dense)               (None, 256)               512       
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 769 (3.00 KB)
Trainable params: 769 (3.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(features, labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
131/131 [==============================] - 5s 9ms/step - loss: 1.5787 - accuracy: 0.9597 - val_loss: 0.5324 - val_accuracy: 0.8161
Epoch 2/10
131/131 [==============================] - 1s 7ms/step - loss: 0.0855 - accuracy: 0.9791 - val_loss: 0.6698 - val_accuracy: 0.7864
Epoch 3/10
131/131 [==============================] - 1s 6ms/step - loss: 0.0544 - accuracy: 0.9806 - val_loss: 0.3438 - val_accuracy: 0.9061
Epoch 4/10
131/131 [==============================] - 1s 6ms/step - loss: 0.0459 - accuracy: 0.9868 - val_loss: 0.5231 - val_accuracy: 0.8630
Epoch 5/10
131/131 [==============================] - 1s 6ms/step - loss: 0.0434 - accuracy: 0.9854 - val_loss: 0.2686 - val_accuracy: 0.9243
Epoch 6/10
131/131 [==============================] - 1s 6ms/step - loss: 0.0388 - accuracy: 0.9871 - val_loss: 0.5629 - val_accuracy: 0.8381
Epoch 7/10
131/131 [==============================] - 1s 6ms/step - loss: 0.0641 - accuracy: 0.9789 - val_loss: 0.8134 - val_accuracy: 0.8295
Epoch 

In [ ]:
model.save('pneumonia.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
